In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request as request

from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

ModuleNotFoundError: No module named 'selenium'

In [ ]:
options = Options()
ua = UserAgent(verify_ssl=False)
userAgent = ua.random
print(userAgent)

In [ ]:
options.add_argument(f'user-agent={userAgent}')
# chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=458511975')

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

In [ ]:
import time

total_lists = []

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=458511975')

time.sleep(1)

page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

time.sleep(1)

songIds = soup.select("#frm > div > table > tbody td:nth-child(4) > div > a")
total_lists = total_lists + songIds

time.sleep(1)

In [ ]:
test = []
for i in range(len(total_lists)):
    test.append(total_lists[i]['href'])

In [ ]:
test1 = []
for i in range(len(test)):
    test1.append(str(test[i]).split('.')[2].split("(")[1].split(")")[0])

In [ ]:
test2 = []
for i in range(len(test1)):
    test2.append(str(test1[i]).strip("''"))

In [ ]:
test2

In [ ]:
# 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수 생성

singer_list = []
singName_list = []
date_list = []
genre_list = []
lyrics_list = []
like_list = []

In [ ]:
# for문으로 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수에 내용 담고 저장하기

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for songId in test2:
    
    driver.get(f'https://www.melon.com/song/detail.htm?songId={songId}')

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    time.sleep(2)

    try:
    
        # 가수명 가져오기
        artist_name = soup.select_one('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1) ')
        singer_list.append(artist_name.string)

        # 노래제목 가져오기
        song_name = soup.select_one("#downloadfrm > div > div > div.entry > div.info > div.song_name ")
        singName_list.append(song_name.text.strip().split('\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t')[1])

        # 발매일 가져오기
        date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)')
        date_list.append(date.string)

        # 장르 가져오기
        genre = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)')
        genre_list.append(genre.string)

        # 가사 가져오기
        lyric = str(soup.select_one('#d_video_summary')).replace('<br/>', ' ').replace(
            '<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->', '').replace('</div>', '').replace('\t', '').replace('\n', '')
        lyric = lyric.replace('  ', ' ')
        lyrics_list.append(lyric)

        # 좋아요수 가져오기
        like = soup.select_one("#d_like_count")
        like_list.append(like.text)

        time.sleep(2)
    except:
        singer_list.append(0)
        singName_list.append(0)
        date_list.append(0)
        lyrics_list.append(0)
        like_list.append(0)

        time.sleep(1)

In [ ]:
ballad = {'가수명':singer_list, '노래제목':singName_list, '발매일':date_list, '장르':genre_list, '가사':lyrics_list, '좋아요수':like_list}

In [ ]:
import pandas as pd

In [ ]:
ballad_calm= pd.DataFrame(ballad)
ballad_calm.head()

In [ ]:
ballad_calm.장르.unique()

In [ ]:
list = ['발라드, 국내영화','발라드, 인디음악','발라드','발라드,록/메탈']
ballad_calm = ballad_calm[ballad_calm['장르'].isin(list)]

In [ ]:
ballad_calm.reset_index(drop=True, inplace=True)

In [ ]:
ballad_calm.drop(columns='장르',inplace=True)

In [ ]:
ballad_calm

In [ ]:
ballad_calm['좋아요수'] = ballad_calm['좋아요수'].str.replace(',','')

In [ ]:
ballad_calm['발매일'] = ballad_calm['발매일'].str.replace('.', '-')

In [ ]:
ballad_calm['발매일'] = pd.to_datetime(ballad_calm['발매일'])

In [ ]:
len(ballad_calm)

In [ ]:
ballad_calm.to_csv('../Data/ballad_calm3.csv', index=False)

In [ ]:
pd.read_csv("../Data/ballad_calm3.csv")